<a href="https://colab.research.google.com/github/SeungukJeong/ML4AI/blob/main/1207_jinjae_Untitled48_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 37.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os 


import numpy as np
import pandas as pd
import glob
import math 

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline 
from sklearn.tree import DecisionTreeRegressor 

from lightgbm import LGBMRegressor 
from xgboost import XGBRegressor 

#import optuna #Optuna: 하이퍼파라미터 최적화 프레임워크
#from optuna.samplers import TPESampler 

import scipy
import scipy.signal

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense 
from sklearn.model_selection import KFold 
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import os 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [6]:
project_path = '/content/drive/MyDrive/FinalProject'

os.chdir(project_path)
os.getcwd()

'/content/drive/MyDrive/ML4AI_22Fall/FinalProject'

In [7]:
train_time = pd.read_csv('submission.csv')

In [8]:
train_list = os.listdir('/content/drive/MyDrive/FinalProject/test')

In [9]:
def create_frame(data, type_data='train'):
    data = data.fillna(0)
    
    data_transform = data.describe().iloc[1:, :] 

    data_transform.loc['skew'] = data.skew().tolist()
    data_transform.loc['mad'] = data.mad().tolist()
    data_transform.loc['kurtosis'] = data.kurtosis().tolist()
            
    data_transform = pd.DataFrame(data_transform.unstack()).reset_index()
    data_transform = data_transform.rename(columns={0: 'value'})
    data_transform['feature'] = data_transform['level_0'] + '_' + data_transform['level_1']
    data_transform = data_transform.drop(['level_0', 'level_1'], axis=1).set_index('feature').T
    
    return data_transform

In [10]:
all_train = pd.DataFrame()

for file in tqdm(train_list):
    df = pd.read_csv('test/' + file)
    df = create_frame(df, type_data='train')
    df['segment_id'] = file[:-4] 
    all_train = all_train.append(df)

all_train = all_train.reset_index(drop=True)

100%|██████████| 4520/4520 [15:29<00:00,  4.86it/s]


In [11]:
all_train

feature,sensor_1_mean,sensor_1_std,sensor_1_min,sensor_1_25%,sensor_1_50%,sensor_1_75%,sensor_1_max,sensor_1_skew,sensor_1_mad,sensor_1_kurtosis,...,sensor_10_std,sensor_10_min,sensor_10_25%,sensor_10_50%,sensor_10_75%,sensor_10_max,sensor_10_skew,sensor_10_mad,sensor_10_kurtosis,segment_id
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2044671025
1,0.117781,214.018525,-1104.0,-138.0,0.0,141.0,1072.0,-0.012138,168.019360,0.538458,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2044757067
2,2.453976,238.403013,-980.0,-155.0,0.0,156.0,1110.0,0.045184,187.363859,0.331740,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2045714087
3,1.838469,379.722993,-2011.0,-239.0,0.0,239.0,2093.0,-0.010816,293.691582,0.921590,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,2045964904
4,0.317228,222.217433,-1045.0,-141.0,0.0,143.0,930.0,-0.020059,173.489323,0.542508,...,788.975468,-4229.0,-474.0,0.0,403.0,2671.0,-0.891582,591.526144,1.574872,2047074908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4515,-2.317978,276.982501,-1220.0,-187.0,0.0,181.0,1504.0,-0.016680,219.341173,0.224009,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,744672527
4516,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,746463163
4517,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,747006133
4518,3.581774,438.148164,-2459.0,-269.0,0.0,283.0,2577.0,-0.022852,338.856249,0.994679,...,1015.201457,-5775.0,-612.0,0.0,616.0,6878.0,0.093129,771.062666,1.701457,746808252


In [12]:
all_train.to_csv('training_stat_add.csv',index=False)